### Jupyter Notebook Demo: Approximating Cosine Similarity based on the Cauchy-Schwarz Inequality

This notebook demonstrates a technique to approximate the cosine similarity
between two L2-normalized vectors by splitting them at a random feature
and calculating the dot product of the prefix and the product of the L2 norms
of the suffix.


**Introduction**

Cosine similarity is a common measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. For L2-normalized vectors $a$ and $b$, the cosine similarity is simply their dot product: $\cos(a, b) = a \cdot b$.

This notebook explores an approximation technique for the cosine similarity. Given two $m$-dimensional L2-normalized vectors $a$ and $b$, we can choose a random feature index $f$ (where $1 \le f \le m-1$) and approximate the cosine similarity using the following inequality:

$$\cos(a, b) \le a^{<f} \cdot b^{<f} + ||a^{\ge f}||_2 ||b^{\ge f}||_2$$

where:
* $a^{<f}$ is a vector with the first $f$ features of $a$ and all subsequent features set to 0, which we call the *prefix* vector.
* $b^{<f}$ is similarly defined for vector $b$.
* $a^{\ge f}$ is a vector with the features from index $f$ to $m-1$ of $a$ and all preceding features set to 0, which we call the *sufix* vector.
* $b^{\ge f}$ is similarly defined for vector $b$.
* $||v||_2$ denotes the L2 norm (Euclidean norm) of a vector $v$.

This inequality is derived from the Cauchy-Schwarz inequality, which states that:

$$a \cdot b \le ||a||_2 ||b||_2.$$

Given that $a$ and $b$ are normalized vectors, the inequality does not help us much, as it simply states that the dot-product of the vectors (and by extension their cosine similarity) is smaller or equal to 1. Well, we already knew that...

However, this approximation can be useful in certain scenarios, especially when dealing with high-dimensional data, sparse data, or when computational efficiency is a concern. This notebook will demonstrate this concept with randomly generated L2-normalized vectors.

In [2]:
import numpy as np
from numpy.linalg import norm

def l2_normalize(vector):
    """L2-normalizes a given vector."""
    norm_val = norm(vector)
    if norm_val == 0:
        return vector
    return vector / norm_val

def approximate_cosine_similarity(a, b, f):
    """
    Approximates the cosine similarity between two L2-normalized vectors
    using a random feature split.

    Args:
        a (np.array): The first L2-normalized vector.
        b (np.array): The second L2-normalized vector.
        f (int): The random feature index (1 <= f <= len(a) - 1).

    Returns:
        float: The upper bound approximation of the cosine similarity.
    """
    m = len(a)
    if not (1 <= f <= m - 1):
        raise ValueError(f"Feature index f must be between 1 and {m-1} (inclusive). Got {f}.")

    a_prefix = np.concatenate((a[:f], np.zeros(m - f)))
    b_prefix = np.concatenate((b[:f], np.zeros(m - f)))

    a_suffix = np.concatenate((np.zeros(f), a[f:]))
    b_suffix = np.concatenate((np.zeros(f), b[f:]))

    dot_product_prefix = np.dot(a_prefix, b_prefix)
    norm_product_suffix = norm(a_suffix) * norm(b_suffix)

    return dot_product_prefix + norm_product_suffix

In [3]:
# Set the dimension of the vectors
m = 100

# Generate two random vectors
np.random.seed(42)  # for reproducibility
vector_a_raw = np.random.rand(m)
vector_b_raw = np.random.rand(m)

# L2-normalize the vectors
vector_a = l2_normalize(vector_a_raw)
vector_b = l2_normalize(vector_b_raw)

# Calculate the actual cosine similarity
cosine_similarity_actual = np.dot(vector_a, vector_b)
print(f"Actual Cosine Similarity: {cosine_similarity_actual:.4f}")

# Choose a random feature index f
f = np.random.randint(1, m)
print(f"Randomly chosen feature index f: {f}")

# Approximate the cosine similarity
cosine_similarity_approx = approximate_cosine_similarity(vector_a, vector_b, f)
print(f"Approximated Cosine Similarity (upper bound): {cosine_similarity_approx:.4f}")

# Verify the inequality
if cosine_similarity_actual <= cosine_similarity_approx + 1e-9: # Adding a small tolerance for floating-point comparisons
    print("The inequality holds: cos(a, b) <= dot(a^<f, b^<f) + ||a^>=f||_2 ||b^>=f||_2")
else:
    print("The inequality DOES NOT hold (potential issue in implementation or theory understanding).")

Actual Cosine Similarity: 0.7210
Randomly chosen feature index f: 24
Approximated Cosine Similarity (upper bound): 0.9269
The inequality holds: cos(a, b) <= dot(a^<f, b^<f) + ||a^>=f||_2 ||b^>=f||_2


**Exploring the Impact of the Split Point**

Let's explore how the choice of the random feature split $f$ can affect the tightness of the upper bound. We will calculate the approximation for several different random values of $f$ and compare them to the actual cosine similarity.

In [4]:
# Number of random splits to test
num_splits = 5

print("\nExploring different random feature splits:")
for i in range(num_splits):
    f_random = np.random.randint(1, m)
    approx_similarity = approximate_cosine_similarity(vector_a, vector_b, f_random)
    print(f"  Split f = {f_random}: Approximated Cosine Similarity = {approx_similarity:.4f}")

print(f"\nActual Cosine Similarity (for reference): {cosine_similarity_actual:.4f}")


Exploring different random feature splits:
  Split f = 75: Approximated Cosine Similarity = 0.7852
  Split f = 72: Approximated Cosine Similarity = 0.7959
  Split f = 36: Approximated Cosine Similarity = 0.8970
  Split f = 38: Approximated Cosine Similarity = 0.8908
  Split f = 84: Approximated Cosine Similarity = 0.7502

Actual Cosine Similarity (for reference): 0.7210


In the previous demo, we explored the inequality:

$$\cos(a, b) \le a^{<f} \cdot b^{<f} + ||a^{\ge f}||_2 ||b^{\ge f}||_2.$$

Two other inequalities that also hold are:

1.  $$\cos(a, b) \le a^{<f} \cdot b^{<f} + ||a^{\ge f}||_2$$
2.  $$\cos(a, b) \le a^{<f} \cdot b^{<f} + ||b^{\ge f}||_2$$

These assume that the other vector has all of their features in the suffix, thus its suffix norm would be 1. We will now generate random L2-normalized vectors and, for a chosen random feature split, calculate the upper bounds provided by all three inequalities and compare them to the actual cosine similarity.

In [5]:
def approximate_cosine_similarity_v2(a, b, f):
    """
    Approximates cosine similarity: dot(a^<f, b^<f) + ||a^>=f||_2
    """
    m = len(a)
    if not (1 <= f <= m - 1):
        raise ValueError(f"Feature index f must be between 1 and {m-1} (inclusive). Got {f}.")

    a_prefix = np.concatenate((a[:f], np.zeros(m - f)))
    b_prefix = np.concatenate((b[:f], np.zeros(m - f)))

    a_suffix = np.concatenate((np.zeros(f), a[f:]))

    dot_product_prefix = np.dot(a_prefix, b_prefix)
    norm_suffix_a = norm(a_suffix)

    return dot_product_prefix + norm_suffix_a

def approximate_cosine_similarity_v3(a, b, f):
    """
    Approximates cosine similarity: dot(a^<f, b^<f) + ||b^>=f||_2
    """
    m = len(a)
    if not (1 <= f <= m - 1):
        raise ValueError(f"Feature index f must be between 1 and {m-1} (inclusive). Got {f}.")

    a_prefix = np.concatenate((a[:f], np.zeros(m - f)))
    b_prefix = np.concatenate((b[:f], np.zeros(m - f)))

    b_suffix = np.concatenate((np.zeros(f), b[f:]))

    dot_product_prefix = np.dot(a_prefix, b_prefix)
    norm_suffix_b = norm(b_suffix)

    return dot_product_prefix + norm_suffix_b

# Test the new approximations
# Calculate the actual cosine similarity
cosine_similarity_actual = np.dot(vector_a, vector_b)
print(f"Actual Cosine Similarity: {cosine_similarity_actual:.4f}")

# Choose a random feature index f
f = np.random.randint(1, m)
print(f"Randomly chosen feature index f: {f}")

# Approximate the cosine similarity using all three versions
approx_similarity_v1 = approximate_cosine_similarity(vector_a, vector_b, f)
approx_similarity_v2 = approximate_cosine_similarity_v2(vector_a, vector_b, f)
approx_similarity_v3 = approximate_cosine_similarity_v3(vector_a, vector_b, f)

print(f"Approximation (Version 1): {approx_similarity_v1:.4f}")
print(f"Approximation (Version 2): {approx_similarity_v2:.4f}")
print(f"Approximation (Version 3): {approx_similarity_v3:.4f}")

# Verify the inequalities
tolerance = 1e-9
if cosine_similarity_actual <= approx_similarity_v1 + tolerance:
    print("Inequality holds for Version 1.")
else:
    print("Inequality DOES NOT hold for Version 1.")

if cosine_similarity_actual <= approx_similarity_v2 + tolerance:
    print("Inequality holds for Version 2.")
else:
    print("Inequality DOES NOT hold for Version 2.")

if cosine_similarity_actual <= approx_similarity_v3 + tolerance:
    print("Inequality holds for Version 3.")
else:
    print("Inequality DOES NOT hold for Version 3.")

Actual Cosine Similarity: 0.7210
Randomly chosen feature index f: 99
Approximation (Version 1): 0.7210
Approximation (Version 2): 0.7378
Approximation (Version 3): 0.8536
Inequality holds for Version 1.
Inequality holds for Version 2.
Inequality holds for Version 3.



More examples of useful cosine similarity approximations can be found in the following papers by David C. Anastasiu:

```bibtex
@inproceedings{anastasiu-icde2014,
   author    = {David C. Anastasiu and George Karypis},
   title     = {L2AP: Fast Cosine Similarity Search With Prefix L-2 Norm Bounds},
   booktitle = {The 30th IEEE International Conference on Data Engineering},
   series    = {ICDE 2014},
   year      = {2014},
   pages     = {784-795},
   location  = {Chicago, USA},
}

@inproceedings{anastasiu-cikm2015,
   author    = {David C. Anastasiu and George Karypis},
   title     = {L2Knng: Fast Exact K-Nearest Neighbor Graph Construction with L2-Norm Pruning},
   booktitle = {Proceedings of the 24th ACM International Conference on Information and Knowledge Management},
   series    = {CIKM'15},
   year      = {2015},
   pages     = {791-800},
   publisher = {ACM},
   address   = {New York, NY, USA},
   location  = {Melbourne, Australia}
}
```
